In [1]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Barra de progreso de un proceso
# ------------------------------------------------------------------------------
from tqdm import tqdm

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

In [3]:
df=pd.read_csv('data/totales_estand_cod.csv',index_col=0)
df.head(3)

,registro,clima,temperatura,sens_termica,humedad,viento,total,fecha_nueva,festividad,festividad_1,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,año_map,estacion_map
0,1,2,-0.828948,-0.678635,1.279463,-0.376545,985,2018-01-01,New Year's Day,1,1,0,0,0,0
1,2,2,-0.722312,-0.740034,0.507498,0.882110,801,2018-01-02,NaN,0,1,1,1,0,0
2,3,1,-1.642047,-1.746977,-1.317148,0.861132,1349,2018-01-03,NaN,0,1,2,1,0,0


In [4]:
df.drop(['registro','fecha_nueva','festividad'], axis=1,inplace=True)
df.columns

Index(['clima', 'temperatura', 'sens_termica', 'humedad', 'viento', 'total',
       'festividad_1', 'mes_bueno', 'dia_semana_nuevo', 'no_laboral_nuevo',
       'año_map', 'estacion_map'],
      dtype='object')

In [5]:
X=df.drop(['total'],axis=1)
y=df['total']

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
# Definir diccionario con los hiperparámetros a modificar
# min_samples_split: número mínimo de observaciones que debe tener un nodo para dividirse, por defecto [10, 50, 100]
# min_samples_leaf: número mínimo de observaciones que debe tener cada uno de los nodos hijos para que se produzca la division, por defecto [10, 50, 100]
param = {"max_depth": [2,4,6], "max_features": [1,2,3], "min_samples_split": [10, 150, 300], "min_samples_leaf": [10,150,300]}

In [9]:
gs_rf = GridSearchCV(
            estimator=RandomForestRegressor(), # tipo de modelo que queremos hacer
            param_grid= param, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [10]:
# ajustamos el modelo que acabamos de definir en el GridSearch
# 🚨 Esta celda puede tardar en ejecutarse

gs_rf.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 4, 6], 'max_features': [1, 2, 3],
                         'min_samples_leaf': [10, 150, 300],
                         'min_samples_split': [10, 150, 300]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [11]:
bosque = gs_rf.best_estimator_
bosque

RandomForestRegressor(max_depth=6, max_features=3, min_samples_leaf=10,
                      min_samples_split=10)

In [12]:
y_pred_test_rf = bosque.predict(x_test)
y_pred_train_rf = bosque.predict(x_train)

In [4]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [14]:
df_totales_encod_estand = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "Random Forest totales encod - estand")
df_totales_encod_estand 

,MAE,MSE,RMSE,R2,set,modelo
0,544.397598,493782.467193,702.696568,0.855739,test,Random Forest totales encod - estand
1,550.731295,542973.596415,736.867421,0.857700,train,Random Forest totales encod - estand


## Sin estandarizar:

In [2]:
df2 = pd.read_csv('data/totales_encod.csv', index_col = 0)

In [3]:
df2.drop(['registro','fecha_nueva','festividad'], axis=1,inplace=True)
df2.columns

Index(['clima', 'temperatura', 'sens_termica', 'humedad', 'viento', 'total',
       'festividad_1', 'mes_bueno', 'dia_semana_nuevo', 'no_laboral_nuevo',
       'año_map', 'estacion_map'],
      dtype='object')

In [5]:
X2=df2.drop(['total'],axis=1)
y2=df2['total']

In [6]:
x2_train, x2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)


In [7]:
# Definir diccionario con los hiperparámetros a modificar
# min_samples_split: número mínimo de observaciones que debe tener un nodo para dividirse, por defecto [10, 50, 100]
# min_samples_leaf: número mínimo de observaciones que debe tener cada uno de los nodos hijos para que se produzca la division, por defecto [10, 50, 100]
param2 = {"max_depth": [16,17], "max_features": [1,2,3], "min_samples_split": [5, 75, 200], "min_samples_leaf": [5,75,200]}

In [9]:
gs_rf2 = GridSearchCV(
            estimator=RandomForestRegressor(), # tipo de modelo que queremos hacer
            param_grid= param2, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [10]:

gs_rf2.fit(x2_train, y2_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [16, 17], 'max_features': [1, 2, 3],
                         'min_samples_leaf': [5, 75, 200],
                         'min_samples_split': [5, 75, 200]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [11]:
bosque2 = gs_rf2.best_estimator_
bosque2

RandomForestRegressor(max_depth=16, max_features=3, min_samples_leaf=5,
                      min_samples_split=5)

In [12]:
y2_pred_test_rf = bosque2.predict(x2_test)
y2_pred_train_rf = bosque2.predict(x2_train)

In [13]:
df_totales_encod = metricas(y2_test, y2_train, y2_pred_test_rf, y2_pred_train_rf, "Random Forest totales encod")
df_totales_encod

,MAE,MSE,RMSE,R2,set,modelo
0,472.161657,382951.911140,618.831084,0.888119,test,Random Forest totales encod
1,421.567713,336874.936827,580.409284,0.911713,train,Random Forest totales encod
